 
 ###  `simd`  and  `declare`   `simd`  Constructs

 The following example illustrates the basic use of the  `simd`  construct  to assure the compiler that the loop can be vectorized.

In [ ]:

/*
* name: SIMD.1c
* type: C
* version: omp_4.0
*/
void star( double *a, double *b, double *c, int n, int *ioff )
{
   int i;
   #pragma omp simd
   for ( i = 0; i < n; i++ )
      a[i] *= b[i] * c[i+ *ioff];
}



In [ ]:

! name: SIMD.1f
! type: F-free
! version: omp_4.0
subroutine star(a,b,c,n,ioff_ptr)
   implicit none
   double precision :: a(*),b(*),c(*)
   integer          :: n, i
   integer, pointer :: ioff_ptr

   !$omp simd
   do i = 1,n
      a(i) = a(i) * b(i) * c(i+ioff_ptr)
   end do

end subroutine



 When a function can be inlined within a loop the compiler has an opportunity to  vectorize the loop. By guaranteeing SIMD behavior of a function's operations,  characterizing the arguments of the function and privatizing temporary  variables of the loop, the compiler can often create faster, vector  for  the loop. In the examples below the  `declare`   `simd`  construct is  used on the  _add1_  and  _add2_  functions to enable creation of their  corresponding SIMD function versions for execution within the associated SIMD  loop. The functions characterize two different approaches of accessing data  within the function: by a single variable and as an element in a data array,  respectively. The  _add3_  C function uses dereferencing.

 The  `declare`   `simd`  constructs also illustrate the use of   `uniform`  and  `linear`  clauses.  The  `uniform(fact)`  clause  indicates that the variable  _fact_  is invariant across the SIMD lanes. In  the  _add2_  function  _a_  and  _b_  are included in the  `uniform`   list because the C pointer and the Fortran array references are constant.  The   _i_  index used in the  _add2_  function is included in a  `linear`   clause with a constant-linear-step of 1, to guarantee a unity increment of the  associated loop. In the  `declare`   `simd`  construct for the  _add3_   C function the   `linear(a,b:1)`  clause instructs the compiler to generate  unit-stride loads across the SIMD lanes; otherwise,  costly {gather}  instructions would be generated for the unknown sequence of access of the  pointer dereferences.

 In the  `simd`  constructs for the loops the  `private(tmp)`  clause is  necessary to assure that the each vector operation has its own  _tmp_   variable.

In [ ]:

/*
* name: SIMD.2c
* type: C
* version: omp_4.0
*/
#include <stdio.h>

#pragma omp declare simd uniform(fact)
double add1(double a, double b, double fact)
{
   double c;
   c = a + b + fact;
   return c;
}

#pragma omp declare simd uniform(a,b,fact) linear(i:1)
double add2(double *a, double *b, int i, double fact)
{
   double c;
   c = a[i] + b[i] + fact;
   return c;
}

#pragma omp declare simd uniform(fact) linear(a,b:1)
double add3(double *a, double *b, double fact)
{
   double c;
   c = *a + *b + fact;
   return c;
}

void work( double *a, double *b, int n )
{
   int i;
   double tmp;
   #pragma omp simd private(tmp)
   for ( i = 0; i < n; i++ ) {
      tmp  = add1( a[i],  b[i], 1.0);
      a[i] = add2( a,     b, i, 1.0) + tmp;
      a[i] = add3(&a[i], &b[i], 1.0);
   }
}

int main(){
   int i;
   const int N=32;
   double a[N], b[N];

   for ( i=0; i<N; i++ ) {
      a[i] = i; b[i] = N-i;
   }

   work(a, b, N );

   for ( i=0; i<N; i++ ) {
      printf("%d %f\n", i, a[i]);
   }

   return 0;
}



In [ ]:

! name: SIMD.2f
! type: F-free
! version: omp_4.0
program main
   implicit none
   integer, parameter :: N=32
   integer :: i
   double precision   :: a(N), b(N)
   do i = 1,N
      a(i) = i-1
      b(i) = N-(i-1)
   end do
   call work(a, b, N )
   do i = 1,N
      print*, i,a(i)
   end do
end program

function add1(a,b,fact) result(c)
   implicit none
!$omp declare simd(add1) uniform(fact)
   double precision :: a,b,fact, c
   c = a + b + fact
end function

function add2(a,b,i, fact) result(c)
   implicit none
!$omp declare simd(add2) uniform(a,b,fact) linear(i:1)
   integer          :: i
   double precision :: a(*),b(*),fact, c
   c = a(i) + b(i) + fact
end function

subroutine work(a, b, n )
   implicit none
   double precision           :: a(n),b(n), tmp
   integer                    :: n, i
   double precision, external :: add1, add2

   !$omp simd private(tmp)
   do i = 1,n
      tmp  = add1(a(i), b(i), 1.0d0)
      a(i) = add2(a,    b, i, 1.0d0) + tmp
      a(i) = a(i) + b(i) + 1.0d0
   end do
end subroutine



 A thread that encounters a SIMD construct executes a vectorized  of the  iterations. Similar to the concerns of a worksharing loop a loop vectorized  with a SIMD construct must assure that temporary and reduction variables are  privatized and declared as reductions with clauses.  The example below  illustrates the use of  `private`  and  `reduction`  clauses in a SIMD  construct.

In [ ]:

/*
* name: SIMD.3c
* type: C
* version: omp_4.0
*/
double work( double *a, double *b, int n )
{
   int i;
   double tmp, sum;
   sum = 0.0;
   #pragma omp simd private(tmp) reduction(+:sum)
   for (i = 0; i < n; i++) {
      tmp = a[i] + b[i];
      sum += tmp;
   }
   return sum;
}



In [ ]:

! name: SIMD.3f
! type: F-free
! version: omp_4.0
subroutine work( a, b, n, sum )
   implicit none
   integer :: i, n
   double precision :: a(n), b(n), sum, tmp

   sum = 0.0d0
   !$omp simd private(tmp) reduction(+:sum)
   do i = 1,n
      tmp = a(i) + b(i)
      sum = sum + tmp
   end do

end subroutine work



 A  `safelen(N)`  clause in a  `simd`  construct assures the compiler that  there are no loop-carried dependencies for vectors of size  _N_  or below. If  the  `safelen`  clause is not specified, then the default safelen value is  the number of loop iterations.   The  `safelen(16)`  clause in the example below guarantees that the vector   is safe for vectors up to and including size 16.  In the loop,  _m_  can  be 16 or greater, for correct  execution.  If the value of  _m_  is less  than 16, the behavior is undefined.

In [ ]:

/*
* name: SIMD.4c
* type: C
* version: omp_4.0
*/
void work( float *b, int n, int m )
{
   int i;
   #pragma omp simd safelen(16)
   for (i = m; i < n; i++)
      b[i] = b[i-m] - 1.0f;
}



In [ ]:

! name: SIMD.4f
! type: F-free
! version: omp_4.0
subroutine work( b, n, m )
   implicit none
   real       :: b(n)
   integer    :: i,n,m

   !$omp simd safelen(16)
   do i = m+1, n
      b(i) = b(i-m) - 1.0
   end do
end subroutine work



 The following SIMD construct instructs the compiler to collapse the  _i_  and   _j_  loops into a single SIMD loop in which SIMD chunks are executed by  threads of the team. Within the workshared loop chunks of a thread, the SIMD  chunks are executed in the lanes of the vector units.

In [ ]:

/*
* name: SIMD.5c
* type: C
* version: omp_4.0
*/
void work( double **a, double **b, double **c, int n )
{
   int i, j;
   double tmp;
   #pragma omp for simd collapse(2) private(tmp)
   for (i = 0; i < n; i++) {
      for (j = 0; j < n; j++) {
         tmp = a[i][j] + b[i][j];
         c[i][j] = tmp;
      }
   }
}



In [ ]:

! name: SIMD.5f
! type: F-free
! version: omp_4.0
subroutine work( a, b, c,  n )
   implicit none
   integer :: i,j,n
   double precision :: a(n,n), b(n,n), c(n,n), tmp

   !$omp do simd collapse(2) private(tmp)
   do j = 1,n
      do i = 1,n
         tmp = a(i,j) + b(i,j)
         c(i,j) = tmp
      end do
   end do

end subroutine work



  ###  `inbranch`  and  `notinbranch`  Clauses

 The following examples illustrate the use of the  `declare`   `simd`   construct with the  `inbranch`  and  `notinbranch`  clauses. The   `notinbranch`  clause informs the compiler that the function  _foo_  is  never called conditionally in the SIMD loop of the function  _myaddint_ . On  the other hand, the  `inbranch`  clause for the function goo indicates that  the function is always called conditionally in the SIMD loop inside  the function  _myaddfloat_ .

In [ ]:

/*
* name: SIMD.6c
* type: C
* version: omp_4.0
*/
#pragma omp declare simd linear(p:1) notinbranch
int foo(int *p){
  *p = *p + 10;
  return *p;
}

int myaddint(int *a, int *b, int n)
{
#pragma omp simd
  for (int i=0; i<n; i++){
      a[i]  = foo(&b[i]);  /* foo is not called under a condition */
  }
  return a[n-1];
}

#pragma omp declare simd linear(p:1) inbranch
float goo(float *p){
  *p = *p + 18.5f;
  return *p;
}

int myaddfloat(float *x, float *y, int n)
{
#pragma omp simd
  for (int i=0; i<n; i++){
     x[i] = (x[i] > y[i]) ? goo(&y[i]) : y[i];
       /* goo is called under the condition (or within a branch) */
  }
  return x[n-1];
}



In [ ]:

! name: SIMD.6f
! type: F-free
! version: omp_4.0
function foo(p) result(r)
  implicit none
!$omp declare simd(foo) notinbranch
  integer :: p, r
  p = p + 10
  r = p
end function foo

function myaddint(a, b,  n) result(r)
  implicit none
  integer :: a(*), b(*), n, r
  integer :: i
  integer, external :: foo

  !$omp simd
  do i=1, n
      a(i) = foo(b(i))  ! foo is not called under a condition
  end do
  r = a(n)

end function myaddint

function goo(p) result(r)
  implicit none
!$omp declare simd(goo) inbranch
  real :: p, r
  p = p + 18.5
  r = p
end function goo

function myaddfloat(x, y, n) result(r)
  implicit none
  real :: x(*), y(*), r
  integer :: n
  integer :: i
  real, external :: goo

  !$omp simd
  do i=1, n
     if (x(i) > y(i)) then
        x(i) = goo(y(i))
        ! goo is called under the condition (or within a branch)
     else
        x(i) = y(i)
     endif
  end do

  r = x(n)
end function myaddfloat



 In the  below, the function  _fib()_  is called in the main program and  also recursively called in the function  _fib()_  within an  `if`   condition. The compiler creates a masked vector version and a non-masked vector  version for the function  _fib()_  while retaining the original scalar  version of the  _fib()_  function.

In [ ]:

/*
* name: SIMD.7c
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <stdlib.h>

#define N 45
int a[N], b[N], c[N];

#pragma omp declare simd inbranch
int fib( int n )
{
   if (n <= 1)
      return n;
   else {
      return fib(n-1) + fib(n-2);
   }
}

int main(void)
{
   int i;

   #pragma omp simd
   for (i=0; i < N; i++) b[i] = i;

   #pragma omp simd
   for (i=0; i < N; i++) {
      a[i] = fib(b[i]);
   }
   printf("Done a[%d] = %d\n", N-1, a[N-1]);
   return 0;
}



In [ ]:

! name: SIMD.7f
! type: F-free
! version: omp_4.0
program fibonacci
   implicit none
   integer,parameter :: N=45
   integer           :: a(0:N-1), b(0:N-1)
   integer           :: i
   integer, external :: fib

   !$omp simd
   do i = 0,N-1
      b(i) = i
   end do

   !$omp simd
   do i=0,N-1
      a(i) = fib(b(i))
   end do

   write(*,*) "Done a(", N-1, ") = ", a(N-1)
                        ! 44  701408733
end program

recursive function fib(n) result(r)
   implicit none
!$omp declare simd(fib) inbranch
   integer  :: n, r

   if (n <= 1) then
      r = n
   else
      r = fib(n-1) + fib(n-2)
   endif

end function fib



 ### Loop-Carried Lexical Forward Dependence

  The following example tests the restriction on an SIMD loop with the loop-carried lexical forward-dependence. This dependence must be preserved for the correct execution of SIMD loops.

 A loop can be vectorized even though the iterations are not completely independent when it has loop-carried dependences that are forward lexical dependences, indicated in the  below by the read of  _A[j+1]_  and the write to  _A[j]_  in C/C++  (or  _A(j+1)_  and  _A(j)_  in Fortran). That is, the read of  _A[j+1]_  (or  _A(j+1)_  in Fortran) before the write to  _A[j]_  (or  _A(j)_  in Fortran) ordering must be preserved for each iteration in  _j_  for valid SIMD  generation.

 This test assures that the compiler preserves the loop carried lexical forward-dependence for generating a correct SIMD .

In [ ]:

/*
* name: SIMD.8c
* type: C
* version: omp_4.0
*/
#include <stdio.h>
#include <math.h>

int   P[1000];
float A[1000];

float do_work(float *arr)
{
  float pri;
  int i;
#pragma omp simd lastprivate(pri)
  for (i = 0; i < 999; ++i) {
    int j = P[i];

    pri = 0.5f;
    if (j % 2 == 0) {
      pri = A[j+1] + arr[i];
    }
    A[j] = pri * 1.5f;
    pri = pri + A[j];
  }
  return pri;
}

int main(void)
{
  float pri, arr[1000];
  int i;

  for (i = 0; i < 1000; ++i) {
     P[i]   = i;
     A[i]   = i * 1.5f;
     arr[i] = i * 1.8f;
  }
  pri = do_work(&arr[0]);
  if (pri == 8237.25) {
    printf("passed: result pri = %7.2f (8237.25) \n", pri);
  }
  else {
    printf("failed: result pri = %7.2f (8237.25) \n", pri);
  }
  return 0;
}



In [ ]:

! name: SIMD.8f
! type: F-free
! version: omp_4.0
module work

integer :: P(1000)
real    :: A(1000)

contains
function do_work(arr) result(pri)
  implicit none
  real, dimension(*) :: arr

  real :: pri
  integer :: i, j

  !$omp simd private(j) lastprivate(pri)
  do i = 1, 999
    j = P(i)

    pri = 0.5
    if (mod(j-1, 2) == 0) then
      pri = A(j+1) + arr(i)
    endif
    A(j) = pri * 1.5
    pri = pri + A(j)
  end do

end function do_work

end module work

program simd_8f
  use work
  implicit none
  real :: pri, arr(1000)
  integer :: i

  do i = 1, 1000
     P(i)   = i
     A(i)   = (i-1) * 1.5
     arr(i) = (i-1) * 1.8
  end do
  pri = do_work(arr)
  if (pri == 8237.25) then
    print 2, "passed", pri
  else
    print 2, "failed", pri
  endif
2 format(a, ": result pri = ", f7.2, " (8237.25)")

end program

